In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv3D, AveragePooling3D, MaxPooling3D, ZeroPadding3D, Dropout, Flatten, Dense, Activation
from keras.optimizers import SGD
from sklearn.feature_extraction.image import extract_patches

Using Theano backend.


In [22]:
import nibabel as nib
import SimpleITK as sitk
import glob
import numpy as np
import os
import datetime

In [21]:
from nipype.interfaces.ants import N4BiasFieldCorrection

In [ ]:
def correct_bias(in_file, out_file=None):
    n4 = N4BiasFieldCorrection()
    n4.inputs.in_file = in_file
    if not out_file:
        out_file = os.path.join(os.path.dirname(in_file), "bias_corrected_" + os.path.basename(in_file))
    n4.inputs.out_file = os.path.baspath(out_file)
    return out_file

In [28]:
def sitk_corrrect_bias(image):
    return sitk.N4BiasFieldCorrection(sitk.Cast(image, sitk.sitkFloat64),
                                      image == image)

In [25]:
def read_image(subject_folder, name):
    return sitk.ReadImage(glob.glob(os.path.join(subject_folder,
                                                 "*" + name + ".*",
                                                 "*" + name + ".*.mha"))[0])

In [24]:
def read_and_combine():
    return read_subject_folder("/home/neuro-user/Downloads/BRATS/BRATS2015_Training/HGG/brats_2013_pat0001_1")
    
def read_subject_folder(folder):
    "Truth is the last element in the data array."
    data = []
    for name in ["T1", "T1c", "Flair", "3more"]:
        image = read_image(folder, name)
        corrected_image = sitk_corrrect_bias(image)
        data.append(sitk.GetArrayFromImage(corrected_image))
    return np.array(data)

In [29]:
d = read_and_combine()

In [3]:
def read_and_combine():
    return read_subject_folder("/home/neuro-user/Downloads/BRATS/BRATS2015_Training/HGG/brats_2013_pat0001_1")
    
def read_subject_folder(folder):
    data = []
    for name in ["T1", "T1c", "Flair", "3more"]:
        image = read_image(folder, name)
        corrected_image = sitk_corrrect_bias(image)
        data.append(sitk.GetArrayFromImage(corrected_image))
    
    flair_image = sitk.ReadImage(glob.glob(os.path.join(folder, "*Flair.*", "*Flair*.mha"))[0])
    t1_image = sitk.ReadImage(glob.glob(os.path.join(folder, "*T1.*", "*T1.*.mha"))[0])
    t1c_image = sitk.ReadImage(glob.glob(os.path.join(folder, "*T1c.*", "*T1c.*.mha"))[0])
    truth_image = sitk.ReadImage(glob.glob(os.path.join(folder, "*3more.*", "*3more.*.mha"))[0])
    return np.array([sitk.GetArrayFromImage(t1_image), 
                     sitk.GetArrayFromImage(t1c_image), 
                     sitk.GetArrayFromImage(flair_image),
                     sitk.GetArrayFromImage(truth_image)])

In [4]:
def train_batch(batch, model):
    x_train = batch[:,:3,:,:]
    y_train = batch[:,3,16,16,16]
    model.train_on_batch(x_train, y_train)

In [5]:
def index_num_2_index(index_number, array_shape):
    index = [0 for i in range(len(array_shape))]
    aaa(index_number, array_shape, -1, index)
    return index

In [6]:
def aaa(x, y, z, index):
    if x > y[z]:
        aaa(x/y[z], y, z-1, index)
    index[z] = np.remainder(x, y[z])

In [7]:
def get_n_patches(patches):
    n_patches = 1
    return product_of_array(patches.shape[:4])


def product_of_array(array):
    x = 1
    for element in array:
        x *= element
    return x

In [15]:
def pop_random(in_list):
    index = np.random.choice(range(len(in_list)))
    return in_list.pop(index)

In [8]:
def train_model(data, model, batch_size=100):

    patches = extract_patches(data, patch_shape=data_patch_shape)
    n_patches = get_n_patches(patches)

    iterations = range(n_patches)
    while len(iterations) > 0:
        indexes_to_process = []
        if len(iterations) > batch_size:

            for i in range(batch_size):
                indexes_to_process.append(pop_random(iterations))
                
        else:
            [indexes_to_process.append(pop_random(iterations)) for i in range(len(iterations))]
        
        batch = []
        for i in indexes_to_process:
            index = index_num_2_index(i, patches.shape)
            batch.append(patches[index[0], index[1], index[2], index[3]])

        train_batch(np.array(batch), model)
        
    return model

In [9]:
subject_dirs = glob.glob("/home/neuro-user/Downloads/BRATS/BRATS2015_Training/*/*/")

In [10]:
n_channels = 3
patch_size = 33
patch_shape = (n_channels, patch_size, patch_size, patch_size)
data_patch_shape = (n_channels + 1, patch_size, patch_size, patch_size)

In [11]:
dim_ordering = 'th'
model = Sequential()
model.add(Conv3D(input_shape=patch_shape, 
                 dim_ordering='th',
                 nb_filter=68,
                 kernel_dim1=7,
                 kernel_dim2=7,
                 kernel_dim3=7,
                 activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(1,1,1)))

model.add(Conv3D(input_shape=patch_shape, 
                 dim_ordering='th',
                 nb_filter=68,
                 kernel_dim1=3,
                 kernel_dim2=3,
                 kernel_dim3=3,
                 activation='relu'))

model.add(Flatten())
model.add(Dense(5, init='normal'))
model.add(Activation('softmax'))
sgd = SGD(lr=0.001, decay=0.01, momentum=0.9)
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd')

In [ ]:
for subject_dir in subject_dirs:
    train_model(read_subject_folder(subject_dir), model)
    date = datetime.datetime.now().date()
    time = datetime.datetime.now().time()
    model_file = "model_{0}{1}{2}_{3}:{4}:{5}.h5".format(date.year, date.month, date.day,
                                                         time.hour, time.minute, time.second)
    model.save(moel_file)

In [17]:
import numpy as np
a = range(10)

In [18]:
np.random.choice(a, replace=False)

7

In [19]:
pop_random(a)

4

In [20]:
a

[0, 1, 2, 3, 5, 6, 7, 8, 9]